## Get listing from RentFaster

In [1]:
import httpx
import pandas as pd
from urllib.parse import urlencode
import asyncio
import nest_asyncio

nest_asyncio.apply()

async def fetch_data():
    url = "https://www.rentfaster.ca/api/map.json"
    headers = {
        "cookie": "RFUUID=0b2cd84e-b1d0-48b0-b875-3bffa61e40f5; PHPSESSID=83de974bcdf5b8e0a88ba17f40b97296",
        "content-type": "application/x-www-form-urlencoded",
        "user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/118.0.0.0 Safari/537.36"
    }

    types = ["Apartment", "Townhouse", "Mobile", "Vacation Home", "Storage", "Room For Rent", "House", "Loft", "Condo Unit", "Basement", "Parking Spot", "Main Floor", "Duplex", "Acreage"]
    city_id = "ab/calgary"

    dataframes = []
    async with httpx.AsyncClient() as client:
        for type_ in types:
            data = {"type[]": type_, "city_id": city_id}
            payload = urlencode(data)
            response = await client.post(url, data=payload, headers=headers)
            df = pd.json_normalize(response.json()['listings'])
            print(f"Number of listing for {type_}: {len(df)}")
            dataframes.append(df)

    final_df = pd.concat(dataframes)
    print(f"Total number of rows: {len(final_df)}")
    return final_df

# Run the function
df = asyncio.run(fetch_data())

Number of listing for Apartment: 383
Number of listing for Townhouse: 405
Number of listing for Mobile: 1
Number of listing for Vacation Home: 0
Number of listing for Storage: 24
Number of listing for Room For Rent: 264
Number of listing for House: 500
Number of listing for Loft: 9
Number of listing for Condo Unit: 500
Number of listing for Basement: 500
Number of listing for Parking Spot: 15
Number of listing for Main Floor: 292
Number of listing for Duplex: 148
Number of listing for Acreage: 4
Total number of rows: 3045


In [2]:
df.drop_duplicates

df.shape

(3045, 33)

## Plot listing on a map (Optional)

In [3]:
import folium

# Create a map centered at some location
m = folium.Map(location=[df['latitude'].mean(), df['longitude'].mean()], zoom_start=10)

# Add points
for idx, row in df.iterrows():
    folium.CircleMarker(
        location=[row['latitude'], row['longitude']],
        radius=1,  # defines the radius of the CircleMarker
        color="blue",
        fill=True,
        fill_color="blue",
        fill_opacity=0.3  # 50% transparency
    ).add_to(m)
# Save it to html
#m.save('map.html')
#m

In [4]:
df.columns

Index(['ref_id', 'id', 'userId', 'phone', 'phone_2', 'email', 'availability',
       'a', 'v', 'f', 's', 'title', 'intro', 'city', 'community', 'latitude',
       'longitude', 'marker', 'link', 'thumb2', 'preferred_contact', 'type',
       'price', 'price2', 'beds', 'beds2', 'sq_feet', 'sq_feet2', 'baths',
       'baths2', 'cats', 'dogs', 'utilities_included'],
      dtype='object')

## Get the first row for testing

In [5]:
# Access the first row using iloc[0]
first_row = df.iloc[0]

# Convert the Series to a dictionary and iterate over its items
for column_name, value in first_row.to_dict().items():
    print(f"{column_name}: {value}")

print(first_row[32])

ref_id: 531446
id: 531446
userId: 1433701
phone: 4036898269
phone_2: 
email: True
availability: Immediate
a: 2000-01-01
v: 1
f: 2
s: RSYNC
title: Downtown Calgary Apartments
intro: 825 8 Avenue SW
city: Calgary
community: Beltline
latitude: 51.0457187
longitude: -114.0805283
marker: 48fe460417812991985db96c922835c3
link: /ab/calgary/rentals/apartment/1-bedroom/beltline/pet-friendly/531446
thumb2: https://f1a3d4fea3a9a877e732-356deb4d9644d2835b7712e712dbd1ea.ssl.cf2.rackcdn.com/531446/thumber_8998700.v.ac5adea583bcdd311300f8b3137a0685.jpg
preferred_contact: Phone||Text||Email
type: Apartment
price: 1995
price2: 2350
beds: 1
beds2: 2
sq_feet: 606
sq_feet2: 890
baths: 1
baths2: 2
cats: 2.0
dogs: 2.0
utilities_included: ['Heat', 'Water']
['Heat', 'Water']


C:\Users\smart\AppData\Local\Temp\ipykernel_10516\689567336.py:8: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  print(first_row[32])


### Reload the modules after making changes to them

In [29]:
import models
#import get_listing
import polygon_module

import importlib
importlib.reload(models)
importlib.reload(polygon_module)
#importlib.reload(get_listing)


<module 'polygon_module' from 'c:\\Users\\smart\\Documents\\Hobby Projects\\2_Web Scraping\\cbe\\polygon_module.py'>

## Testing database functions

In [7]:
def store_df_to_db(df, db):
    db.deactivate_old_listings()
    for _, row in df.iterrows():
        db.insert_listing(row)

# Usage:
# Create an instance of Listing_db
#listing_db = Listing_db()

# DataFrame df is assumed to exist
#store_df_to_db(df, listing_db)

Close connection to prevent database being locked.

Troubleshooting:

In [8]:
school_db = models.School_db()

listing_db = models.Listing_db()

In [9]:
#school_db.con.close()

In [10]:
school_ids = school_db.get_all_school_ids()

In [11]:
all_listing_ids_coordinates = listing_db.get_all_listing_ids_coordinates()

In [12]:
#listing_db.con.commit()